In [1]:
from keras import applications, optimizers
from keras.layers import Dropout, Flatten, Dense, Input
from keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import keras
import numpy as np

Using TensorFlow backend.


In [2]:
# dimensions of our images.
img_width, img_height = 224, 224

train_data_dir = '../dataset/train'
test_data_dir = '../dataset/test'
validation_data_dir = '../dataset/validation'
nb_train_samples = 1792
nb_test_samples = 210
nb_validation_samples = 525

In [3]:
def buildModel():
    # build the VGG16 network
    base_model = applications.VGG16(weights='imagenet', 
                                    include_top=False,
                                    input_tensor=Input(shape=(img_width, img_height, 3)))

    for layer in base_model.layers:
        layer.trainable = False
        
    top_model = base_model.output
    top_model = Flatten(name="Flatten")(top_model)
    top_model = Dense(512, activation='relu')(top_model)
    top_model = Dense(256, activation='relu')(top_model)
    top_model = Dense(6, activation='softmax')(top_model)
    
    model = Model(inputs=base_model.input, outputs=top_model)

    #     model.summary()
    
    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    
    return model

In [4]:
def generateData(batch_size):
    train_datagen = ImageDataGenerator(
        horizontal_flip=True,
        rescale=1. /255,
        rotation_range=30,
        shear_range=0.2,
        zoom_range=0.2)

    validation_datagen = ImageDataGenerator(rescale=1. /255)

    train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        shuffle=True,
        class_mode='categorical')

    validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical')
    
    return [train_generator,validation_generator]

In [5]:
def fineTuneModel(model, train_generator, validation_generator, epochs, batch_size):
    model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples // batch_size)
    
    return model

In [6]:
def getMetrics(model, batch_size):
    
    test_datagen = ImageDataGenerator(rescale=1. /255)

    test_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_width,img_height),
        shuffle=False,
        batch_size=1,
        class_mode='categorical')
    
    predicted_results = model.predict_generator(test_generator, 
                                                steps = nb_test_samples)
    predicted_results = np.argmax(predicted_results, axis=1)
    targets = ["cardboard", "glass", "metal", "paper", "plastic", "trash"]
    
    # confusion matrix
    print("CONFUSION MATRIX:")
    print(confusion_matrix(test_generator.classes, predicted_results))
    
    # classification report
    print("CLASSIFICATION REPORT:")
    print(classification_report(test_generator.classes, predicted_results, target_names=targets))

In [7]:
def runExperiments():
    batch_sizes = [8,16,32]
    epochs = [10,20,30]
    config = 1
    
    for batch_size in batch_sizes:
        for epoch in epochs:
            
            print("*************** Test ", config, " Batch size: ", batch_size, " Epochs: ", epoch, "***************")           
            keras.backend.clear_session()
            
            model = buildModel()
            train_generator,test_generator = generateData(batch_size)
            trained_model = fineTuneModel(model,train_generator, test_generator, epoch, batch_size)
            metrics = getMetrics(trained_model, batch_size)
            
            config += 1

In [8]:
runExperiments()

W0803 15:19:32.201968 139791472756480 deprecation_wrapper.py:119] From /home/martin/anaconda3/envs/martin/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:95: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.

W0803 15:19:32.204038 139791472756480 deprecation_wrapper.py:119] From /home/martin/anaconda3/envs/martin/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:98: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0803 15:19:32.222422 139791472756480 deprecation_wrapper.py:119] From /home/martin/anaconda3/envs/martin/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:102: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0803 15:19:32.223049 139791472756480 deprecation_wrapper.py:119] From /home/martin/anaconda3/envs/martin/lib/python3.6/site-packages/keras/backend/tensorflow_backend.p

*************** Test  1  Batch size:  8  Epochs:  10 ***************


W0803 15:19:33.286929 139791472756480 deprecation_wrapper.py:119] From /home/martin/anaconda3/envs/martin/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



Found 1792 images belonging to 6 classes.
Found 525 images belonging to 6 classes.


W0803 15:19:33.591894 139791472756480 deprecation.py:323] From /home/martin/anaconda3/envs/martin/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10
224/224 [==============================] - 593s 3s/step - loss: 1.5086 - acc: 0.4693 - val_loss: 1.0890 - val_acc: 0.5846
Epoch 2/10
224/224 [==============================] - 569s 3s/step - loss: 0.9993 - acc: 0.6222 - val_loss: 0.7833 - val_acc: 0.7176
Epoch 3/10
224/224 [==============================] - 566s 3s/step - loss: 0.7872 - acc: 0.7081 - val_loss: 0.7042 - val_acc: 0.7331
Epoch 4/10
224/224 [==============================] - 571s 3s/step - loss: 0.7114 - acc: 0.7349 - val_loss: 0.6912 - val_acc: 0.7215
Epoch 5/10
224/224 [==============================] - 569s 3s/step - loss: 0.6962 - acc: 0.7310 - val_loss: 0.8116 - val_acc: 0.7041
Epoch 6/10
224/224 [==============================] - 567s 3s/step - loss: 0.6322 - acc: 0.7573 - val_loss: 0.6240 - val_acc: 0.7621
Epoch 7/10
224/224 [==============================] - 567s 3s/step - loss: 0.5830 - acc: 0.7801 - val_loss: 0.6393 - val_acc: 0.7660
Epoch 8/10
224/224 [==============================] - 562s 3s/step - 

Epoch 19/30
224/224 [==============================] - 554s 2s/step - loss: 0.2963 - acc: 0.8956 - val_loss: 0.5902 - val_acc: 0.8008
Epoch 20/30
224/224 [==============================] - 554s 2s/step - loss: 0.2863 - acc: 0.8945 - val_loss: 0.7000 - val_acc: 0.8066
Epoch 21/30
224/224 [==============================] - 553s 2s/step - loss: 0.2727 - acc: 0.9035 - val_loss: 0.6086 - val_acc: 0.7969
Epoch 22/30
224/224 [==============================] - 551s 2s/step - loss: 0.2440 - acc: 0.9169 - val_loss: 0.8462 - val_acc: 0.7930
Epoch 23/30
224/224 [==============================] - 552s 2s/step - loss: 0.2271 - acc: 0.9185 - val_loss: 0.7223 - val_acc: 0.8085
Epoch 24/30
224/224 [==============================] - 552s 2s/step - loss: 0.2493 - acc: 0.9090 - val_loss: 0.6340 - val_acc: 0.8143
Epoch 25/30
224/224 [==============================] - 552s 2s/step - loss: 0.2042 - acc: 0.9286 - val_loss: 0.7231 - val_acc: 0.7892
Epoch 26/30
224/224 [==============================] - 552s 2s

Found 1792 images belonging to 6 classes.
Found 525 images belonging to 6 classes.
Epoch 1/30
112/112 [==============================] - 560s 5s/step - loss: 1.6625 - acc: 0.4799 - val_loss: 0.7117 - val_acc: 0.7676
Epoch 2/30
112/112 [==============================] - 512s 5s/step - loss: 0.9170 - acc: 0.6590 - val_loss: 0.7640 - val_acc: 0.7092
Epoch 3/30
112/112 [==============================] - 516s 5s/step - loss: 0.7654 - acc: 0.7249 - val_loss: 0.7009 - val_acc: 0.7544
Epoch 4/30
112/112 [==============================] - 516s 5s/step - loss: 0.6665 - acc: 0.7561 - val_loss: 0.6120 - val_acc: 0.7760
Epoch 5/30
112/112 [==============================] - 515s 5s/step - loss: 0.5883 - acc: 0.7773 - val_loss: 0.7676 - val_acc: 0.7525
Epoch 6/30
112/112 [==============================] - 513s 5s/step - loss: 0.5417 - acc: 0.8092 - val_loss: 0.6300 - val_acc: 0.7898
Epoch 7/30
112/112 [==============================] - 511s 5s/step - loss: 0.4836 - acc: 0.8309 - val_loss: 0.6156 - va